## finetuning
### huggingface上にあるLLMをinstallしてきてfinetuning

In [2]:
## training dataの収納

import os
import pandas as pd
import json

# ファイルパスの設定

for dataset in ['train','val']:
    txt_folder_path = f"../radnlp_2024_train_val_20240731/ja/main_task/{dataset}/"  # TXTファイルが保存されているフォルダ
    csv_file_path = f"../radnlp_2024_train_val_20240731/ja/main_task/{dataset}/label.csv"  # ラベルファイル

    # ラベルCSVファイルを読み込む
    label_data = pd.read_csv(csv_file_path)

    # ファインチューニング用データを格納するリスト
    finetune_data = []

    # フォルダ内のTXTファイルを処理
    for txt_file in os.listdir(txt_folder_path):
        if txt_file.endswith(".txt"):
            file_id = txt_file.split(".")[0]  # ファイル名からIDを取得
            file_path = os.path.join(txt_folder_path, txt_file)

            # 対応するラベルを検索
            matching_label = label_data[label_data["id"] == int(file_id)]
            if not matching_label.empty:
                # テキストを読み込む
                with open(file_path, 'r', encoding='utf-8') as file:
                    content = file.read().strip()

                # ラベルを取得
                t = matching_label["t"].values[0]
                n = matching_label["n"].values[0]
                m = matching_label["m"].values[0]

                # ファインチューニング用データを追加
                finetune_data.append({
                    "prompt": content,
                    "completion": f"{t} {n} {m}"
                })

    # ファインチューニング用データをJSONL形式で保存
    output_file = f"../finetune_jsons/finetune_dataset_{dataset}.jsonl"
    with open(output_file, 'w', encoding='utf-8') as jsonl_file:
        for entry in finetune_data:
            json.dump(entry, jsonl_file, ensure_ascii=False)
            jsonl_file.write('\n')

    print(f"ファインチューニング用データが {output_file} に保存されました！")


ファインチューニング用データが ../finetune_jsons/finetune_dataset_train.jsonl に保存されました！
ファインチューニング用データが ../finetune_jsons/finetune_dataset_val.jsonl に保存されました！


In [1]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import Dataset
from accelerate import Accelerator

# GPUの確認
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# モデルとトークナイザーのロード
model_name = "EQUES/MedLLama3-JP-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",  # Multi-GPU対応
    offload_folder="mnt/hdd1/offload",  # オフロード用ディレクトリ
    offload_state_dict=True,
    torch_dtype=torch.bfloat16  # BFloat16の有効化
)
model.gradient_checkpointing_enable()  # 勾配チェックポイントを有効化してメモリを節約

# ファインチューニングデータの準備
def prepare_dataset(jsonl_file_path):
    data = []
    with open(jsonl_file_path, 'r', encoding='utf-8') as jsonl_file:
        for line in jsonl_file:
            entry = json.loads(line.strip())
            data.append({
                "prompt": entry["prompt"],
                "completion": entry["completion"]
            })
    return Dataset.from_list(data)

# データセットのロード
jsonl_file_path_train = "../finetune_jsons/finetune_dataset_train.jsonl"
dataset_train = prepare_dataset(jsonl_file_path_train)
jsonl_file_path_val = "../finetune_jsons/finetune_dataset_val.jsonl"
dataset_val = prepare_dataset(jsonl_file_path_val)

with open('../tnm_prompt.txt', 'r', encoding='utf-8') as file:
    tnm_prompt_text = file.read()

tnm_prompt_text_base = (
    "あなたは優秀な医師です。以下の文章に基づき肺癌に関して常に正しい判断ができます。"
    "進行度分類は以下のTNM第８版に準拠しています。何も言わずに以下を覚え、与えられた文章からよく考えてTNM分類を選んでください。\n\n"
)
tnm_prompt = (
    f"{tnm_prompt_text_base}\n\n"
    f"{tnm_prompt_text}\n\n"
    "以下の文章を読んで、TNM分類を正確に選択し、必ず以下の形式で出力してください：\n"
    "T<number>[optional_letter] N<number>[optional_letter] M<number>[optional_letter]\n\n"
)
tokenizer.pad_token = tokenizer.eos_token

def preprocess_function(examples):
    inputs = [f"{tnm_prompt}{prompt} 出力：" for prompt in examples['prompt']]
    targets = [completion for completion in examples['completion']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = dataset_train.map(preprocess_function, batched=True).remove_columns(["prompt", "completion"])
eval_dataset = dataset_val.map(preprocess_function, batched=True).remove_columns(["prompt", "completion"])

# TrainingArgumentsの設定
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    gradient_accumulation_steps=8,  # 勾配蓄積を有効化してメモリ消費を削減
    bf16=True,  # BFloat16を使用
    fp16=False,  # FP16を無効化（BFloat16を使用するため）
    report_to="none",  # ログレポート先を無効化
)

# Trainerの初期化
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# トレーニングの実行
trainer.train()
trainer.save_model("../finetuned_model")
tokenizer.save_pretrained("../finetuned_model")



/home/jubuntu/anaconda3/envs/merlin/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/jubuntu/anaconda3/envs/merlin/lib/python3.9/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Using device: cuda


Map: 100%|██████████| 54/54 [00:00<00:00, 3221.57 examples/s]
/home/jubuntu/anaconda3/envs/merlin/lib/python3.9/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_2484606/692654080.py:90: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 0/130 [00:00<?, ?it/s]`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacty of 47.43 GiB of which 139.88 MiB is free. Including non-PyTorch memory, this process has 46.50 GiB memory in use. Of the allocated memory 45.83 GiB is allocated by PyTorch, and 364.62 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF